In [1]:
using JuMP, Gurobi, CSV

In [2]:
pixels = CSV.read("output_ghana_10km.csv", header = 1)
pixelsSize = size(pixels)   #tuple representing dimensions of country
xMax = pixelsSize[1]
yMax = pixelsSize[2]
print(xMax, ", ", yMax)

71, 54

In [3]:
# xMax = 36
# yMax = 27
# pixelsSize = pixelsSize[36:71, 28:54]

In [4]:
totalPop = sum(sum(pixels[i,j] for i in 1:xMax) for j in 1:yMax)
percentage = 0.95
print(totalPop)

2.740868427257601e7

In [5]:
model = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [6]:
@variable(model, S[1:xMax, 1:yMax], Bin)   # S[i,j] = 1 if there is a small access pt at (i,j)
@variable(model, L[1:xMax, 1:yMax], Bin)   # L[i,j] = 1 if there is a large access pt at (i,j)
@variable(model, R[1:xMax, 1:yMax], Bin);

In [7]:
TS = sum(sum(S[i,j] for j in 1:yMax) for i in 1:xMax)   # total number of small access pts
TL = sum(sum(L[i,j] for j in 1:yMax) for i in 1:xMax);  # total number of large access pts

In [8]:
@objective(model, Min, 150*TS+250*TL);   # minimize total cost of access pts

In [9]:
for i in 1:xMax
    for j in 1:yMax
        @constraint(model, R[i,j] <= S[i, j]
                                     +
                                     sum(sum(L[i+dx, j+dy]
                                     for dy in max(trunc(Int, -sqrt(1^2-dx^2)), 1-j) : min(yMax-j, trunc(Int, sqrt(1^2-dx^2))))
                                     for dx in max(-1, 1-i) : min(1, xMax-i)))
    end
end                           

In [10]:
@constraint(model, sum(sum(pixels[i,j]*R[i,j] for i in 1:xMax) for j in 1:yMax) >= percentage*totalPop);

In [11]:
# Solving the optimization problem  
optimize!(model)

Academic license - for non-commercial use only
Optimize a model with 3835 rows, 11502 columns and 29117 nonzeros
Variable types: 0 continuous, 11502 integer (11502 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+07, 3e+07]
Found heuristic solution: objective 324000.00000
Found heuristic solution: objective 159000.00000
Presolve removed 1305 rows and 3907 columns
Presolve time: 0.02s
Presolved: 2530 rows, 7595 columns, 19956 nonzeros
Found heuristic solution: objective 199350.00000
Found heuristic solution: objective 155050.00000
Variable types: 0 continuous, 7595 integer (7595 binary)

Root relaxation: objective 7.950889e+04, 5980 iterations, 0.62 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 79508.8940    0 1507 155050.000 79508.8940  48.7%     -    0s


In [12]:
# Printing the optimal solutions obtained
S_opt = value.(S)
L_opt = value.(L)
println("S = ", S_opt)
println("L = ", L_opt)
println("objective value = ", objective_value(model))

S = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 0.0 0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 -0.0 0.0 -0.0 -0.0 0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 -0.0 0.0 -0.0 -0.0 -0.0 0.0 0.0 -0.0 0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 0.0 -0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 -0.0 -0.0 -0.0 0.0 -0.0 -0.0 -0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 0.0 0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 0.0 -0.0 0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0; -0.0 0.0 0.0 0.0 0.0 -0.0 -0.0 -0.0 -0.0 -0.0 0.0 -0.0 -0.0 0.0 -0.0 -0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0

L = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

In [13]:
for i in 1:xMax
    for j in 1:yMax
        if S_opt[i, j] > 0.0
            println("S:", S_opt[i, j])
            println(i, ", ", j)
        end
        if L_opt[i, j] > 0.0
            println("L:", L_opt[i, j])    
            println(i, ", ", j)
        end
    end
end

L:1.0
5, 39
L:1.0
6, 37
L:1.0
6, 41
L:1.0
7, 35
L:1.0
8, 39
L:1.0
8, 43
L:1.0
9, 33
L:1.0
9, 36
L:1.0
9, 41
L:1.0
9, 45
L:1.0
9, 52
L:1.0
10, 5
L:1.0
10, 31
L:1.0
10, 38
L:1.0
10, 43
L:1.0
11, 3
L:1.0
11, 29
L:1.0
11, 34
L:1.0
11, 40
L:1.0
11, 46
L:1.0
12, 6
L:1.0
12, 11
L:1.0
12, 32
L:1.0
12, 36
L:1.0
12, 42
L:1.0
12, 48
L:1.0
12, 52
L:1.0
13, 4
L:1.0
13, 9
L:1.0
13, 14
L:1.0
13, 28
L:1.0
13, 30
L:1.0
13, 34
L:1.0
13, 39
L:1.0
13, 44
L:1.0
14, 2
L:1.0
14, 7
L:1.0
14, 12
L:1.0
14, 32
L:1.0
14, 36
L:1.0
14, 41
L:1.0
14, 49
L:1.0
15, 5
L:1.0
15, 10
L:1.0
15, 18
L:1.0
15, 29
L:1.0
15, 38
L:1.0
15, 43
L:1.0
15, 47
L:1.0
15, 52
L:1.0
16, 13
L:1.0
16, 16
L:1.0
16, 35
L:1.0
16, 40
L:1.0
16, 45
L:1.0
16, 50
L:1.0
17, 11
L:1.0
17, 27
L:1.0
17, 32
L:1.0
17, 37
L:1.0
17, 42
L:1.0
17, 53
L:1.0
18, 9
L:1.0
18, 29
L:1.0
18, 34
L:1.0
18, 39
L:1.0
18, 44
L:1.0
19, 31
L:1.0
19, 36
L:1.0
19, 41
L:1.0
19, 46
L:1.0
19, 51
L:1.0
20, 10
L:1.0
20, 33
L:1.0
20, 38
L:1.0
20, 43
L:1.0
20, 48
L:1.0
20, 53
L:1.0
